In [682]:
import os
from dotenv import load_dotenv
load_dotenv('key.env')  

key_string = os.getenv('open_ai_API_Key')

In [683]:
# !pip install pydantic --upgrade


In [684]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool, Tool
from langchain_openai import ChatOpenAI

from langchain.tools import tool

In [685]:

from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from typing import Optional


In [686]:
class Extract_toolArgs(BaseModel):
    a: str = Field(description="Company name")
    b: str = Field(description="item")
    c: str = Field(description="Year_1")
    d: str = Field(description="Year_2")
    e :Optional[str] = Field(default=None,description="category")

In [687]:
class Extracted_toolArgs(BaseModel):
    year : list = Field(description="List of years")
    value : list = Field(discription = "List of Values")

In [688]:
import json
from typing import List, Tuple

@tool(args_schema=Extract_toolArgs)
def Extract_Tool(a: str, b: str, c: str, d: str, e: str = None) -> Extract_toolArgs:
    """Extracts specific data for a company from Year_1 to Year_2 based on item and optional category."""
    with open("data_soo.json", "r") as json_file:
        data = json.load(json_file)

    a = a.upper()
    b = b.lower()

    # Filter data for the specific item and category (if provided)
    filtered_data = [
        item for item in data.get(a, [])
        if item["Item"].lower() == b and (not e or item.get("Category", "").lower() == e.lower())
    ]

    if not filtered_data:
        raise ValueError(f"No data found for company '{a}', item '{b}', and category '{e}'.")

    json_data = filtered_data[0]

    year_data = []
    cash_data = []

    # Ensure c and d represent years in the correct order
    if int(c) > int(d):
        c, d = d, c

    for year in range(int(c), int(d) + 1):
        value = json_data.get(str(year))
        if value is not None:
            cash_data.append(value)
            year_data.append(year)

    if len(cash_data) == 0:
        raise ValueError(f"Data for the years {c} to {d} is not available.")

    return cash_data, year_data

In [689]:
class GrowthToolArgs(BaseModel):
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of sales data")


In [690]:
@tool(args_schema=GrowthToolArgs)
def Growth_Tool(sales_data: list, year_data: list) -> list[float]:
    """
    Calculates year-on-year (YoY) growth for 'Net Sales' for a specific company.
    This function receives year data and sales data list.

    Arguments:
    - sales_data: List of corresponding sales.
    - year_data: List of years corresponding to the sales data.
    
    Returns:
    - A list of floats representing the YoY growth percentage for each year compared to the previous year.
    """
    # Ensure there is sufficient data for year-on-year calculation
    if len(sales_data) < 2:
        return ["Not enough data to calculate year-on-year sales growth."]
    
    yoy_growth = []
    
    # Sort the sales data by year to ensure proper calculation
    sales_data_sorted = sorted(zip(year_data, sales_data), key=lambda x: x[0])

    # Extract sorted years and sales
    sorted_years, sorted_sales = zip(*sales_data_sorted)

    for i in range(1, len(sorted_years)):
        current_year = sorted_years[i]
        previous_year = sorted_years[i - 1]

        current_year_sales = sorted_sales[i]
        previous_year_sales = sorted_sales[i - 1]

        if previous_year_sales is not None and current_year_sales is not None:
            # Calculate YoY growth
            growth = ((current_year_sales - previous_year_sales) / previous_year_sales) * 100
            yoy_growth.append(growth)
        else:
            yoy_growth.append(None)  # If data is missing
    
    return yoy_growth


In [691]:
class GrossMarginToolArgs(BaseModel):
    gross_margin_data : list = Field(description="list of Gross margin data")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of Gross margin data")

In [692]:
@tool(args_schema=GrossMarginToolArgs)
def GrossmmarginpercTool(gross_margin_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates percentage of Gross Margin.
    This function receives year data and sales data list and Gross margin.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    -gross_margin_data: List of corresponding gross margin data.
    
    Returns:
    - A list of floats representing the YoY gross margin percentage for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(gross_margin_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, gross_margin_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_gross_margins, sorted_sales = zip(*data_sorted)

    yoy_gross_margin_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            gross_margin_percentage = (sorted_gross_margins[i] / sorted_sales[i]) * 100
            yoy_gross_margin_percentage.append(gross_margin_percentage)
        else:
            yoy_gross_margin_percentage.append(None)  # If data is missing or invalid

    return yoy_gross_margin_percentage

In [693]:
class expensespercArgs(BaseModel):
    t_o_e_data : list = Field(description="list of total operating expenses data")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of total operating expenses data")

In [694]:
@tool(args_schema=expensespercArgs)
def ExpensespercTool(t_o_e_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates expenses percentage sales.
    This function receives year data and sales data list and total operating expenses.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    - t_o_e_data: List of corresponding total operating expenses data.
    
    Returns:
    - A list of floats representing the  expenses percentage sales for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(t_o_e_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, t_o_e_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_t_o_e_data, sorted_sales = zip(*data_sorted)

    yoy_expenses_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            expenses_percentage = (sorted_t_o_e_data[i] / sorted_sales[i]) * 100
            yoy_expenses_percentage.append(expenses_percentage)
        else:
            yoy_expenses_percentage.append(None)  # If data is missing or invalid

    return yoy_expenses_percentage

In [695]:
class TaxratesArgs(BaseModel):
    Provision_of_IT_data : list = Field(description="list of provision for income taxes data")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of total operating expenses data")

In [696]:
@tool(args_schema=TaxratesArgs)
def TaxrateTool(Provision_of_IT_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates tax rate percentage.
    This function receives year data and sales data list and provision for income tax.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    - Provision_of_IT_data: List of corresponding provision for income tax.
    
    Returns:
    - A list of floats representing the tax rate percentage for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(Provision_of_IT_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, Provision_of_IT_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_IT_data, sorted_sales = zip(*data_sorted)

    yoy_Tax_Rate_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            tax_percentage = (sorted_IT_data[i] / sorted_sales[i]) * 100
            yoy_Tax_Rate_percentage.append(tax_percentage)
        else:
            yoy_Tax_Rate_percentage.append(None)  # If data is missing or invalid

    return yoy_Tax_Rate_percentage

In [697]:
class EBTArgs(BaseModel):
    IT_data : list = Field(description="list of income before provision for income taxes")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of total operating expenses data")

In [698]:
@tool(args_schema=EBTArgs)
def EBTTool(IT_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates EBT percentage.
    This function receives year data and sales data list and income before provision for income taxes.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    - IT_data: List of corresponding income before provision for income taxes.
    
    Returns:
    - A list of floats representing the tax rate percentage for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(IT_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, IT_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_IT_data, sorted_sales = zip(*data_sorted)

    yoy_EBT_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            EBT_percentage = (sorted_IT_data[i] / sorted_sales[i]) * 100
            yoy_EBT_percentage.append(EBT_percentage)
        else:
            yoy_EBT_percentage.append(None)  # If data is missing or invalid

    return yoy_EBT_percentage

In [699]:
class NETMarginArgs(BaseModel):
    net_income_data : list = Field(description="list of net income")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years")

In [700]:
@tool(args_schema=NETMarginArgs)
def NetMarginTool(net_income_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates Net margin.
    This function receives year data and sales data list and net income.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    - IT_data: List of corresponding net income.
    
    Returns:
    - A list of floats representing the tax rate percentage for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(net_income_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, net_income_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_net_income_data, sorted_sales = zip(*data_sorted)

    yoy_net_margin_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            net_margin_percentage = (sorted_net_income_data[i] / sorted_sales[i]) * 100
            yoy_net_margin_percentage.append(net_margin_percentage)
        else:
            yoy_net_margin_percentage.append(None)  # If data is missing or invalid

    return yoy_net_margin_percentage

In [701]:
class EPSToolArgs(BaseModel):
    EPS_data : list = Field(description="list of item:{earnings per share} category:{basic}")
    year_data:list = Field(description="list of years of earnings per share")

In [702]:
@tool(args_schema=EPSToolArgs)
def EPS_Tool(EPS_data: list, year_data: list) -> list[float]:
    """
    Calculates year-on-year (YoY) growth for item:'earnings per share' category:'basic' for a specific company.
    This function receives year data and 'earnings per share''basic'.

    Arguments:
    - EPS_data: List of 'earnings per share' 'basic'.
    - year_data: List of corresponding years.
    
    Returns:
    - A list of floats representing the YoY growth percentage for each year compared to the previous year.
    """
    # Ensure there is sufficient data for year-on-year calculation
    if len(EPS_data) < 2:
        return ["Not enough data to calculate year-on-year sales growth."]
    
    yoy_growth = []
    
    # Sort the sales data by year to ensure proper calculation
    sales_data_sorted = sorted(zip(year_data, EPS_data), key=lambda x: x[0])

    # Extract sorted years and sales
    sorted_years, sorted_sales = zip(*sales_data_sorted)

    for i in range(1, len(sorted_years)):
        current_year = sorted_years[i]
        previous_year = sorted_years[i - 1]

        current_year_sales = sorted_sales[i]
        previous_year_sales = sorted_sales[i - 1]

        if previous_year_sales is not None and current_year_sales is not None:
            # Calculate YoY growth
            growth = ((current_year_sales - previous_year_sales) / previous_year_sales) * 100
            yoy_growth.append(growth)
        else:
            yoy_growth.append(None)  # If data is missing
    
    return yoy_growth

In [703]:
class RnDPercArgs(BaseModel):
    RND_data : list = Field(description="list of item:{operating expenses} category:{research and development}")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years")

In [704]:
@tool(args_schema=RnDPercArgs)
def RNDTool(RND_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates research and development percentage sales for  item:'operating expenses' category:'research and development'
    This function receives year data and sales data list and  item:'operating expenses' category:'research and development'.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    - RND_data: list of item:{operating expenses} category:{research and development}
    
    Returns:
    - A list of floats representing the research and development percentage sales for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(RND_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, RND_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_RND_data, sorted_sales = zip(*data_sorted)

    yoy_RND_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            net_margin_percentage = (sorted_RND_data[i] / sorted_sales[i]) * 100
            yoy_RND_percentage.append(net_margin_percentage)
        else:
            yoy_RND_percentage.append(None)  # If data is missing or invalid

    return yoy_RND_percentage

In [705]:
class SNGPercArgs(BaseModel):
    SNG_data : list = Field(description="list of item:{operating expenses} category:{selling, general and administrative}")
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years")

In [706]:
@tool(args_schema=SNGPercArgs)
def SNGTool(SNG_data: list, year_data: list, sales_data: list) -> list[float]:
    """
    Calculates selling and general percentage sales for  item:'operating expenses' category:'selling, general and administrative'
    This function receives year data and sales data list and  item:'operating expenses' category:'selling, general and administrative'.

    Arguments:
    - sales_data: List of corresponding sales data.
    - year_data: List of years corresponding to the sales data.
    - SNG_data: list of item:{operating expenses} category:{selling, general and administrative}
    
    Returns:
    - A list of floats representing the selling and general percentage sales for each year.
    """
    # Ensure there is sufficient data for year-on-year calculation

    if len(SNG_data) != len(sales_data) or len(sales_data) != len(year_data):
        raise ValueError("All input lists must have the same length.")

    # Ensure there is sufficient data for year-on-year calculation
    if len(year_data) < 2:
        raise ValueError("At least two years of data are required for YoY calculation.")

    # Sort the data by year to ensure proper calculation
    data_sorted = sorted(zip(year_data, SNG_data, sales_data), key=lambda x: x[0])

    # Extract sorted years, gross margins, and sales
    sorted_years, sorted_SNG_data, sorted_sales = zip(*data_sorted)

    yoy_SNG_percentage = []

    for i in range(len(sorted_years)):
        if sorted_sales[i] is not None and sorted_sales[i] > 0:  # Avoid division by zero or invalid sales data
            # Calculate Gross Margin percentage for the year
            net_margin_percentage = (sorted_SNG_data[i] / sorted_sales[i]) * 100
            yoy_SNG_percentage.append(net_margin_percentage)
        else:
            yoy_SNG_percentage.append(None)  # If data is missing or invalid

    return yoy_SNG_percentage

In [707]:
class RNDgrowthToolArgs(BaseModel):
    RNDgrowth_data : list = Field(description="list of research and development percentage sales")
    year_data:list = Field(description="list of years of research and development percentage sales")

In [708]:
@tool(args_schema=RNDgrowthToolArgs)
def RNDGrowth_Tool(RNDgrowth_data: list, year_data: list) -> list[float]:
    """
    Calculates year-on-year (YoY) growth for research and development percentage sales for a specific company.
    This function receives year data and research and development percentage sales.

    Arguments:
    - RNDgrowth_data: List of research and development percentage sales.
    - year_data: List of corresponding years.
    
    Returns:
    - A list of floats representing the YoY growth percentage for each year compared to the previous year.
    """
    # Ensure there is sufficient data for year-on-year calculation
    if len(RNDgrowth_data) < 2:
        return ["Not enough data to calculate year-on-year sales growth."]
    
    yoy_growth = []
    
    # Sort the sales data by year to ensure proper calculation
    growth_data_sorted = sorted(zip(year_data, RNDgrowth_data), key=lambda x: x[0])

    # Extract sorted years and sales
    sorted_years, sorted_sales = zip(*growth_data_sorted)

    for i in range(1, len(sorted_years)):
        current_year = sorted_years[i]
        previous_year = sorted_years[i - 1]

        current_year_sales = sorted_sales[i]
        previous_year_sales = sorted_sales[i - 1]

        if previous_year_sales is not None and current_year_sales is not None:
            # Calculate YoY growth
            growth = ((current_year_sales - previous_year_sales) / previous_year_sales) * 100
            yoy_growth.append(growth)
        else:
            yoy_growth.append(None)  # If data is missing
    
    return yoy_growth

In [709]:
import pandas as pd
import os
import io
from langchain.tools import tool
from pydantic import BaseModel, Field

# Tool schema for CSV generation
class GenerateCSVToolArgs(BaseModel):
    data: list = Field(description="List of data (list of dictionaries) to save as a CSV.")
    filename: str = Field(description="Desired filename for the CSV (without extension).")

@tool(args_schema=GenerateCSVToolArgs)
def GenerateCSV_Tool(data: list, filename: str) -> str:
    """
    Generates a CSV file from the provided data and returns the path to download the file.
    """
    try:
        # Create a DataFrame from the data
        df = pd.DataFrame(data)
        
        # Define the path to save the CSV file
        file_path = f"/tmp/{filename}.csv"
        
        # Save the DataFrame as a CSV file
        df.to_csv(file_path, index=False)

        # Return the path to download the file
        return file_path
    except Exception as e:
        return f"Error generating CSV: {str(e)}"


In [710]:
tools = [Extract_Tool,Growth_Tool, GrossmmarginpercTool,ExpensespercTool, TaxrateTool,EBTTool,NetMarginTool,EPS_Tool,RNDTool, SNGTool,RNDGrowth_Tool,GenerateCSV_Tool]

In [711]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=key_string,max_tokens=5000)

In [712]:
prompt = hub.pull("hwchase17/structured-chat-agent")

/Users/gunik/anaconda3/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [713]:
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True)

In [714]:
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

In [715]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,  # Use the conversation memory to maintain context
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
    max_iterations=50
)

In [716]:
initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: Extract_Tool,Growth_Tool, GrossmmarginpercTool,ExpensespercTool, TaxrateTool,EBTTool,NetMarginTool,EPS_Tool,RNDTool, SNGTool,RNDGrowth_Tool. Always return an answer in string.If you get any list convert it into string.Give the downloadable link as it is"
memory.chat_memory.add_message(SystemMessage(content=initial_message))


In [717]:
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    # Add the user's message to the conversation memory
    memory.chat_memory.add_message(HumanMessage(content=user_input))

    # Invoke the agent with the user input and the current chat history
    response = agent_executor.invoke({"input": user_input})
    print("Bot:", response["output"])

    # Add the agent's response to the conversation memory
    memory.chat_memory.add_message(AIMessage(content=response["output"]))



> Entering new AgentExecutor chain...
```
{
  "action": "Extract_Tool",
  "action_input": {
    "a": "Apple",
    "b": "Net Sales",
    "c": "2018",
    "d": "2022"
  }
}
```([265595, 260174, 274515, 365817, 394328], [2018, 2019, 2020, 2021, 2022])```
{
  "action": "Extract_Tool",
  "action_input": {
    "a": "Apple",
    "b": "Gross Margin",
    "c": "2018",
    "d": "2022"
  }
}
```([101839, 98392, 104956, 152836, 170782], [2018, 2019, 2020, 2021, 2022])```
{
  "action": "Extract_Tool",
  "action_input": {
    "a": "Apple",
    "b": "Total Operating Expenses",
    "c": "2018",
    "d": "2022"
  }
}
```([30941, 34462, 38668, 43887, 51345], [2018, 2019, 2020, 2021, 2022])```
{
  "action": "Extract_Tool",
  "action_input": {
    "a": "Apple",
    "b": "Provision for Income Taxes",
    "c": "2018",
    "d": "2022"
  }
}
```([13372, 10481, 9680, 14527, 19300], [2018, 2019, 2020, 2021, 2022])```
{
  "action": "Extract_Tool",
  "action_input": {
    "a": "Apple",
    "b": "Net Income",
  

In [718]:
# !pip install pydantic==1.10.11



In [719]:
# prompt = hub.pull("hwchase17/openai-tools-agent")

In [720]:
# agent = create_tool_calling_agent(
#     llm=llm,  # Language model to use
#     tools=tools,  # List of tools available to the agent
#     prompt=prompt,  # Prompt template to guide the agent's responses
# )

In [721]:
# agent_executor = AgentExecutor.from_agent_and_tools(
#     agent=agent,  # The agent to execute
#     tools=tools,  # List of tools available to the agent
#     verbose=True,  # Enable verbose logging
#     handle_parsing_errors=True,  # Handle parsing errors gracefully
# )

In [722]:
# response = agent_executor.invoke({"input": " extract net sales of Apple 2019 to 2023 "})

In [723]:
# response = agent_executor.invoke({"input": "generate some insights of sales growth of apple"})

In [724]:
# print(response)